In [27]:
import os
import pandas as pd

# Local files
import codebooks
from llm import openrouter_request


# Measure constructs using Large Language Models

# Set API key and choose model
- API key is associated to your account. You can add money to use paid models. 
- All models: https://openrouter.ai/rankings
- Paid models:
    - 'gpt-4.1'
    - 'gpt-4.1-mini'
    - 'anthropic/claude-3.5-sonnet'
- Free models: https://openrouter.ai/models?max_price=0
    - free models: certain requests per minute (e.g., 20) and N requests per day (depends on model). See https://openrouter.ai/docs/limits
    - "google/gemini-2.0-flash-exp:free"
    - "meta-llama/llama-3.1-405b-instruct:free"
    - "meta-llama/llama-3.1-70b-instruct:free"
    - "meta-llama/llama-3.1-8b-instruct:free"


In [30]:
try:
    import api_keys # local file
    OPENROUTER_API_KEY = api_keys.openrouter_key 
except:
    print("Module 'api_key' not found. Add your key here. ") # get it from OpenRouter AI: https://openrouter.ai/settings/keys
    OPENROUTER_API_KEY = "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" # This key will disactivate if you place it on a public repository. Get your own and add some dollars from OpenRouter AI: https://openrouter.ai/settings/keys

# Choose a model
model  = "gpt-4.1" # "gpt-4o", "gpt-4o-mini" (cheaper), "meta-llama/llama-3.1-405b-instruct:free"   others: https://openrouter.ai/models

# Design a prompt
- consider using a different scale: 0 = Not at all, 1 = Mildly related, 2 = Som, 3 = Prototypical mention of the category
- consider changing "mentions" to "expresses" or "is suffering"

In [34]:
# This prompt just asks for a score for each construct

prompt_template_000 = """We asked participants of our study to describe a narrative of a negative event that happened to them. Classify whether the following constructs are clearly present in the participant's narrative (provided below). I'll provide some definitions and examples for the constructs: 

{codebook_string}

{document_type_name}: 
{document}

Structure your response in the following JSON format (just JSON, no extra text), providing a score for the construct:

{{
ID_001: {{'construct_a': score, 'construct_b': score, ...}},
ID_002: {{'construct_a': score, 'construct_b': score, ...}},
...
}}

JSON:
"""



In [42]:

prompt_000 = prompt_template_000.format(codebook_string = codebooks.raya_001,
              document_type_name= 'NARRATIVE',
              document = "ID_9143: No one cares about me. I go to therapy, but it doesnt work. It wont get better. I want out. Im feeling hyperactive."
              )

print('Prompt:')
print(prompt_000)

Prompt:
We asked participants of our study to describe a narrative of a negative event that happened to them. Classify whether the following constructs are clearly present in the participant's narrative (provided below). I'll provide some definitions and examples for the constructs: 


engagement_check: the narrative contains emotional content about a negative event or emotion (and not neutral or irrelevant narratives). 
Your response options:
0 = Absent or irrelevant
1 = Reference to concepts related to failure or letting people down
2 = Reference to an unpleasant experience
3 = Reference to a situation associated with negative emotion
4 = Direct naming of a negative emotion

explicit_mention_of_guilt_or_shame: the narrative contains emotional content about a negative event or emotion (and not neutral or irrelevant narratives).
Your response options:
0 = None (No clear explicit mention of guilt or shame)
1 = Guilt / Guilty
2 = Shame / Ashamed
3 = Synonym for guilt/shame (e.g., regret,

# Submit prompt to openrouter


In [38]:
final_result, metadata = openrouter_request(prompt_000, OPENROUTER_API_KEY, model = model, temperature=0)


In [39]:
final_result

{'ID_9143': {'engagement_check': 4,
  'explicit_mention_of_guilt_or_shame': 0,
  'guilt_negative_behavior_evaluation': 0,
  'guilt_repair_attempt': 0,
  'guilt_other_focused_emphasis': 0,
  'shame_negative_self_evaluation': 2,
  'shame_withdrawal_response': 2,
  'shame_self_focused_distress': 3,
  'anger_blame_externalization': 0,
  'anger_response_expression': 0,
  'anger_unfairness_focus': 0,
  'sadness_self_world_disconnection': 3,
  'sadness_behavioral_response': 2,
  'sadness_loss_helplessness_focus': 3}}

In [43]:
metadata

{'id': 'gen-1750964568-jnVkxHiDNnM0TnYlaOeb',
 'provider': 'OpenAI',
 'model': 'openai/gpt-4.1',
 'object': 'chat.completion',
 'created': 1750964568,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n  "ID_9143": {\n    "engagement_check": 4,\n    "explicit_mention_of_guilt_or_shame": 0,\n    "guilt_negative_behavior_evaluation": 0,\n    "guilt_repair_attempt": 0,\n    "guilt_other_focused_emphasis": 0,\n    "shame_negative_self_evaluation": 2,\n    "shame_withdrawal_response": 2,\n    "shame_self_focused_distress": 3,\n    "anger_blame_externalization": 0,\n    "anger_response_expression": 0,\n    "anger_unfairness_focus": 0,\n    "sadness_self_world_disconnection": 3,\n    "sadness_behavioral_response": 2,\n    "sadness_loss_helplessness_focus": 3\n  }\n}',
    'refusal': None,
    'reasoning': None}}],
 'system_fingerprint': 'fp_51e1070cf2',
 'usage': {'prompt_tokens':

# Loop through documents

In [51]:
df = pd.read_csv('./data/input/DARE Narratives.csv')
df

,ID,Narrative,engagement_check,explicit_mention_of_guilt_or_shame,guilt_negative_behavior_evaluation,guilt_repair_attempt,guilt_other_focused_emphasis,shame_negative_self_evaluation,shame_withdrawal_response,shame_self_focused_distress,anger_blame_externalization,anger_response_expression,anger_unfairness_focus,sadness_self_world_disconnection,sadness_behavioral_response,sadness_loss_helplessness_focus
0,8001,My friend Ian and I went to the Department of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8002,I was visiting home after 6 months and me and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8003,Failures are a part and parcel of life. Someti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8004,Honestly there are so many options. Usually th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8005,"During the second summer of my freshman year, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,8146,My perception of letting people down is a bit ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,8147,"So, I’m taking this massive National Board of ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,8148,This semester I was co directing an original p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,8149,I let my mom down when I decided not to visit ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# dict from ID and narrative columns
texts = dict(zip(df['ID'], df['Narrative']))
texts


{8001: 'My friend Ian and I went to the Department of Motor Vehicles ( Registry of Motor Vehicles in Massachusetts ) together. My California license was lost and his license was expired. We live in the same apartment and planned to go together the next morning. Because I get up much earlier than him, we decided to meet at my office before continuing to the bus stop and then the DMV together. We met at the appointed time, got breakfast at a corner grocery store and then got on the bus. We arrived at our stop, got off, and walked about 15 minutes to the DMV. When we got there, we filled out paperwork, then stood in line. As we waited, people came around to ask why we were there. When someone came to me and I told her I wanted a Massachusetts license, she asked if I had my California driving record. I said no.',
 8002: "I was visiting home after 6 months and me and mother plan to go shopping as well as get some crucial work done at the bank. Since I was coming from a different state ( pla

In [54]:
df.shape

(150, 16)

In [78]:
import tqdm
# Loop through texts and apply openrouter
verbose = False # print all the prompts and responses. 
results = {}
metadata_all = {}

test = False # SET THIS TO TRUE TO RUN ON A SUBSET OF THE DATA FIRST 

# add tqdm to show progress
i = 0
for file, text  in tqdm.tqdm(texts.items()):
    if test:
        if i == 3:
            break
        i += 1
    
    prompt_000 = prompt_template_000.format(codebook_string = codebooks.raya_001,
                document_type_name= 'NARRATIVE',
                document = f'{file}: {text}'
                )
    if verbose:
        print(prompt_000)
    
    # Make request. WARNING: this costs some money depending on model
    response, metadata = openrouter_request(prompt_000, OPENROUTER_API_KEY, model = model, temperature=0)
    results.update(response)
    metadata_all[file]=metadata
    if verbose:
        print('---------------------')
        print(response)
        print('\n========================\n')


  2%|▏         | 3/150 [00:05<04:49,  1.97s/it]


In [79]:
results

{'8001': {'engagement_check': 0,
  'explicit_mention_of_guilt_or_shame': 0,
  'guilt_negative_behavior_evaluation': 0,
  'guilt_repair_attempt': 0,
  'guilt_other_focused_emphasis': 0,
  'shame_negative_self_evaluation': 0,
  'shame_withdrawal_response': 0,
  'shame_self_focused_distress': 0,
  'anger_blame_externalization': 0,
  'anger_response_expression': 0,
  'anger_unfairness_focus': 0,
  'sadness_self_world_disconnection': 0,
  'sadness_behavioral_response': 0,
  'sadness_loss_helplessness_focus': 0},
 '8002': {'engagement_check': 3,
  'explicit_mention_of_guilt_or_shame': 3,
  'guilt_negative_behavior_evaluation': 1,
  'guilt_repair_attempt': 0,
  'guilt_other_focused_emphasis': 1,
  'shame_negative_self_evaluation': 1,
  'shame_withdrawal_response': 0,
  'shame_self_focused_distress': 1,
  'anger_blame_externalization': 1,
  'anger_response_expression': 1,
  'anger_unfairness_focus': 0,
  'sadness_self_world_disconnection': 1,
  'sadness_behavioral_response': 0,
  'sadness_loss

# Turn JSON results into a csv
With each document as a row and each category as a column

In [ ]:
results_df = pd.DataFrame(results).T


In [74]:
results_df

,engagement_check,explicit_mention_of_guilt_or_shame,guilt_negative_behavior_evaluation,guilt_repair_attempt,guilt_other_focused_emphasis,shame_negative_self_evaluation,shame_withdrawal_response,shame_self_focused_distress,anger_blame_externalization,anger_response_expression,anger_unfairness_focus,sadness_self_world_disconnection,sadness_behavioral_response,sadness_loss_helplessness_focus
8001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8002,3,3,1,0,1,1,0,1,2,1,0,1,0,1
8003,1,0,1,0,1,0,0,0,0,0,0,0,0,0
8004,4,1,1,1,1,1,2,2,0,0,0,2,2,2
8005,3,3,1,2,2,0,0,1,0,0,0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146,1,3,2,0,1,1,2,1,0,0,0,1,2,1
8147,3,3,1,2,1,1,1,2,1,1,1,2,1,2
8148,3,3,1,1,1,1,1,1,1,1,1,1,0,1
8149,1,3,1,0,1,0,0,0,0,0,0,1,1,1


In [73]:
results_df.to_csv('./data/output/dare_narratives_gpt-4-1_results.csv', index=False)